# <b>PHASE 1 </b> — DATA CLEANING

## 1. Importing Required Libraries


Pandas and NumPy are imported to make it possible to load the raw dataset, handle missing values, clean messy columns, and perform numerical operations such as calculating medians and detecting outliers during the data cleaning process.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt


## 2. Loading the Raw Dataset


The raw CSV file is loaded into a Pandas DataFrame so that all cleaning, inspection, and transformation operations can be applied to the dataset within Python.


In [2]:
df = pd.read_csv(r"C:\Users\HP\Desktop\Python\messy_food_1000rows.csv")

## 3. Checking the Size of the Dataset


This is used to confirm how many rows and columns exist in the dataset before cleaning, helping to understand the scale of the data and to later verify that no unexpected data loss occurs during the cleaning process.


In [3]:
df.shape

(1050, 12)

## 4. Inspecting Data Structure and Data Types


This is used to check the data types of each column and to identify which columns contain missing values, which helps guide decisions on how each column should be cleaned or converted.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             1050 non-null   int64  
 1   Food           951 non-null    object 
 2   Calories       942 non-null    float64
 3   Price          963 non-null    float64
 4   Category       935 non-null    object 
 5   Restaurant     923 non-null    object 
 6   Location       911 non-null    object 
 7   Date           1050 non-null   object 
 8   Rating         912 non-null    float64
 9   Discount       891 non-null    float64
 10  Delivery_Time  886 non-null    float64
 11  Notes          882 non-null    object 
dtypes: float64(5), int64(1), object(6)
memory usage: 98.6+ KB


## 5. Checking for Missing Values
This code checks each column in the DataFrame `df` for missing (null) values. `df.isnull()` returns `True` for null values, and `.sum()` counts how many nulls are in each column. This helps identify columns that may need cleaning or imputation.


In [5]:
df.isnull().sum()

ID                 0
Food              99
Calories         108
Price             87
Category         115
Restaurant       127
Location         139
Date               0
Rating           138
Discount         159
Delivery_Time    164
Notes            168
dtype: int64

## 6. Removing Extra Spaces from Column Names
This code removes any leading or trailing spaces from the column names in the DataFrame `df`. Using `df.columns.str.strip()` ensures that column names are clean and standardized, which helps avoid errors when referencing them later in the analysis.


In [ ]:
df.columns = df.columns.str.strip()

## 7. Converting 'Rating' to Numeric
This line converts the 'Rating' column to numeric values. Using `pd.to_numeric` with `errors='coerce'` turns any non-numeric values into `NaN`, and `.astype('float64')` ensures the data type is a float. This is important for accurate calculations and analysis on the 'Rating' column.


In [7]:
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').astype('float64')
df['Date'] = pd.to_datetime(df['Date'])

## 8. Displaying DataFrame Information
This line shows a concise summary of the DataFrame `df` using `df.info()`. It displays the number of entries, column names, non-null counts, and data types for each column. This helps quickly understand the structure of the dataset and identify columns that may need cleaning or type conversion.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             1050 non-null   int64         
 1   Food           951 non-null    object        
 2   Calories       942 non-null    float64       
 3   Price          963 non-null    float64       
 4   Category       935 non-null    object        
 5   Restaurant     923 non-null    object        
 6   Location       911 non-null    object        
 7   Date           1050 non-null   datetime64[ns]
 8   Rating         912 non-null    float64       
 9   Discount       891 non-null    float64       
 10  Delivery_Time  886 non-null    float64       
 11  Notes          882 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 98.6+ KB


## 9. Cleaning and Filling Missing Values
This block handles missing values and cleans the data:

1. **Numeric columns**: All numeric columns are filled with their **median** values to handle missing data.
2. **Specific numeric columns** (`Rating`, `Discount`, `Delivery_Time`, `Calories`, `Price`): Missing values are also filled with the **median**.
3. **Categorical columns** (`Food`, `Category`, `Restaurant`, `Location`): Converted to string type and missing values are filled with the **mode** (most frequent value) to maintain consistency.
4. **Notes column**: Missing values are filled with `'No notes'`, extra spaces are removed, and text is capitalized for readability.
5. **Standardizing categorical columns**: Removes extra spaces and capitalizes each word to ensure uniform formatting.


In [9]:
''' 
num_col = df.select_dtypes(include='number').columns
df[num_col] = df[num_col].fillna(df[num_col].median()) 
'''

cols = ['Rating' , 'Discount' , 'Delivery_Time' , 'Calories' , 'Price']
for col in cols:
    df[col] = df[col].fillna(df[col].median())

cat_col = ['Food' , 'Category' , 'Restaurant' , 'Location']
for cat in cat_col:
    df[cat] = df[cat].astype('string').fillna(df[cat].mode().iloc[0])

df['Notes'] = df['Notes'].fillna('No notes').str.strip().str.title()

df[cat_col] = df[cat_col].apply(lambda x: x.str.strip().str.title())

## 10. Additional Data Cleaning and Feature Engineering
This block makes corrections and creates new features:

1. **Fixing Restaurant Name**: Corrects the capitalization of `"Domino'S"` to `"Domino's"` in the 'Restaurant' column.
2. **Creating Fast Delivery Flag**: Adds a new column `'Fast_Delivery'` where orders with `Delivery_Time` ≤ 30 minutes are marked `'True'`. The column is then formatted to have capitalized text for readability.
3. **Calculating Final Price**: Creates a new column `'Final_Price'` by subtracting the `'Discount'` from the `'Price'`, giving the actual amount paid.


In [10]:
df['Restaurant'] = df['Restaurant'].replace("Domino'S","Domino's" , regex=False)

df['Fast_Delivery'] = (df['Delivery_Time'] <= 30.0).astype(str)
df['Fast_Delivery'] = df['Fast_Delivery'].str.lower().str.capitalize()

df['Final_Price'] = df['Price'] - df['Discount']

## 11. Checking and Removing Duplicates
This block handles duplicate rows in the DataFrame:

1. **Check for Duplicates**: `df.duplicated()` identifies rows that are exact duplicates of previous rows.
2. **Remove Duplicates**: `df.drop_duplicates(inplace=True)` removes all duplicate rows from the DataFrame, keeping only the first occurrence. This ensures that the dataset is clean and each record is unique.


In [11]:
df.duplicated()

df.drop_duplicates(inplace=True)


## 12. Saving the Cleaned Dataset
This line saves the cleaned DataFrame `df` to a CSV file. The file path is specified as `"C:\Users\HP\Desktop\Python\my_cleaned_data.csv"`. Using `index=False` ensures that the DataFrame’s index is not saved to the file. This step preserves your cleaned and processed data for future analysis or sharing.


In [12]:
df.to_csv(r"C:\Users\HP\Desktop\Python\my_cleaned_python_data.csv", index=False)

In [13]:
df.head(20)

,ID,Food,Calories,Price,Category,Restaurant,Location,Date,Rating,Discount,Delivery_Time,Notes,Fast_Delivery,Final_Price
0,1,Cake,150.0,400.0,Fast Food,Mcdonald,Abuja,2024-04-07,4.0,20.0,30.0,Excellent,True,380.0
1,2,Salad,500.0,450.0,Grain,Mcdonald,Ibadan,2025-03-18,5.0,10.0,20.0,Excellent,True,440.0
2,3,Sushi,300.0,500.0,Seafood,Mcdonald,Kano,2023-07-06,5.0,15.0,20.0,Late,True,485.0
3,4,Burger,300.0,1200.0,Vegetable,Mcdonald,Port Harcourt,2025-01-01,3.0,0.0,45.0,Delayed,False,1200.0
4,5,Cake,350.0,200.0,Dessert,Kfc,Lagos,2024-03-04,1.0,0.0,25.0,Excellent,True,200.0
5,6,Sandwich,500.0,500.0,Seafood,Subway,Abuja,2025-05-07,5.0,0.0,25.0,Late,True,500.0
6,7,Pizza,350.0,550.0,Fast Food,Pizza Hut,Abuja,2023-08-25,5.0,15.0,20.0,Excellent,True,535.0
7,8,Cake,500.0,300.0,Bakery,Pizza Hut,Lagos,2025-02-09,2.0,5.0,45.0,Late,False,295.0
8,9,Sushi,350.0,350.0,Vegetable,Burger King,Ibadan,2025-01-27,5.0,20.0,45.0,Poor,False,330.0
9,10,Burger,150.0,400.0,Fast Food,Subway,Lagos,2023-05-16,2.0,15.0,60.0,Average,False,385.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             1000 non-null   int64         
 1   Food           1000 non-null   string        
 2   Calories       1000 non-null   float64       
 3   Price          1000 non-null   float64       
 4   Category       1000 non-null   string        
 5   Restaurant     1000 non-null   string        
 6   Location       1000 non-null   string        
 7   Date           1000 non-null   datetime64[ns]
 8   Rating         1000 non-null   float64       
 9   Discount       1000 non-null   float64       
 10  Delivery_Time  1000 non-null   float64       
 11  Notes          1000 non-null   object        
 12  Fast_Delivery  1000 non-null   object        
 13  Final_Price    1000 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2), string(4)
memory usage:

# <b>PHASE 2 </b> — EXPLORATORY DATA ANALYSIS (EDA) 

## 1. Business Size (KPI Metrics)

In [15]:
# Total Revenue Genrated
total_revenue = df['Final_Price'].sum()
total_revenue

np.float64(454245.0)

In [ ]:
# Total Orders
total_orders = df.shape[0]
total_orders

1000

In [17]:
# Average Order Value
average_order_value = df['Final_Price'].mean()
average_order_value

np.float64(454.245)

In [18]:
# Average Ratings
average_rating = df['Rating'].mean()
average_rating

np.float64(3.421)

## 2. Sales Performance

### (i). Revenue by Restaurant

In [19]:
revenue_by_restaurant = df.groupby('Restaurant')['Final_Price'].sum().sort_values(ascending=False)
revenue_by_restaurant

Restaurant
Mcdonald       118100.0
Subway          95470.0
Domino's        64190.0
Pizza Hut       60255.0
Burger King     58695.0
Kfc             57535.0
Name: Final_Price, dtype: float64

### (ii). Revenue by Category

In [20]:
revenue_by_category = df.groupby('Category')['Final_Price'].sum().sort_values(ascending=False)
revenue_by_category

Category
Fast Food    124065.0
Dessert      107500.0
Vegetable     61250.0
Bakery        56590.0
Grain         56220.0
Seafood       48620.0
Name: Final_Price, dtype: float64

### (iii). Revenue by Location

In [21]:
revenue_by_location = df.groupby('Location')['Final_Price'].sum().sort_values(ascending=False)
revenue_by_location

Location
Abuja            174850.0
Lagos            114975.0
Port Harcourt     57855.0
Ibadan            56925.0
Kano              49640.0
Name: Final_Price, dtype: float64

## 3. Customer Behavior
•	Does fast delivery improve ratings?

•	Do discounts increase sales?

•	Do expensive meals get lower ratings?


In [22]:
fast_delivery_behavior = (
    df.groupby('Fast_Delivery')
      .agg(
          Orders=('ID', 'count'),
          Avg_Rating=('Rating', 'mean'),
          Avg_Delivery_Time=('Delivery_Time', 'mean'),
          Avg_Price=('Final_Price', 'mean'),
          Total_Revenue=('Final_Price', 'sum')
      )
)

fast_delivery_behavior


,Orders,Avg_Rating,Avg_Delivery_Time,Avg_Price,Total_Revenue
Fast_Delivery,,,,,
False,276,3.344203,53.043478,454.438406,125425.0
True,724,3.450276,23.087017,454.171271,328820.0


## 4. Pricing & Profitability


How much revenue is lost to discounts?


In [23]:
df['revenue_lost'] = df['Price'] - df['Final_Price']
total_lost = df['revenue_lost'].sum()
total_lost

np.float64(9855.0)

Which category gives highest final revenue?

In [24]:
category_revenue = df.groupby('Category')['Final_Price'].sum().sort_values(ascending=False)
category_revenue

Category
Fast Food    124065.0
Dessert      107500.0
Vegetable     61250.0
Bakery        56590.0
Grain         56220.0
Seafood       48620.0
Name: Final_Price, dtype: float64

Which restaurant is underpricing?

In [25]:
overall_avg_price = df['Final_Price'].mean()
restaurant_avg_price = df.groupby('Restaurant')['Final_Price'].mean()

underpricing_restaurants = restaurant_avg_price[restaurant_avg_price < overall_avg_price]
underpricing_restaurants

Restaurant
Mcdonald    447.348485
Subway      411.508621
Name: Final_Price, dtype: float64

## 5. Time Analysis
<b>Sales by month</b>


In [26]:
df['month'] = df['Date'].dt.month_name()
sales_per_month = df.groupby('month')['Final_Price'].sum()
sales_per_month 

month
April        30600.0
August       41725.0
December     34395.0
February     26875.0
January      36490.0
July         46320.0
June         42990.0
March        41845.0
May          39940.0
November     31720.0
October      34475.0
September    46870.0
Name: Final_Price, dtype: float64

In [ ]:
month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [28]:
monthly_sales = sales_per_month.reindex(month_order)
monthly_sales


month
January      36490.0
February     26875.0
March        41845.0
April        30600.0
May          39940.0
June         42990.0
July         46320.0
August       41725.0
September    46870.0
October      34475.0
November     31720.0
December     34395.0
Name: Final_Price, dtype: float64

<b>Sales Trends (Over Time) </b>

In [29]:
# Monthly trend using Year-Month
df['Year'] = df['Date'].dt.to_period('Y')

sales_trend = df.groupby('Year')['Final_Price'].sum()

sales_trend


Year
2023    156430.0
2024    148700.0
2025    149115.0
Freq: Y-DEC, Name: Final_Price, dtype: float64

<b>Busy vs Slow Periods</b>

Option A: By Month

In [30]:
avg_monthly_sales = monthly_sales.mean()

busy_months = monthly_sales[ monthly_sales > avg_monthly_sales ]
slow_months = monthly_sales[ monthly_sales > avg_monthly_sales ]
busy_months, slow_months

(month
 March        41845.0
 May          39940.0
 June         42990.0
 July         46320.0
 August       41725.0
 September    46870.0
 Name: Final_Price, dtype: float64,
 month
 March        41845.0
 May          39940.0
 June         42990.0
 July         46320.0
 August       41725.0
 September    46870.0
 Name: Final_Price, dtype: float64)

Option B: By Day of Week

In [31]:
df['day'] = df['Date'].dt.day_name()
daily_sales = df.groupby('day')['Final_Price'].sum()
day_order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
daily_sales = daily_sales.reindex(day_order)
daily_sales

day
Sunday       69720.0
Monday       72665.0
Tuesday      59830.0
Wednesday    60815.0
Thursday     66740.0
Friday       65575.0
Saturday     58900.0
Name: Final_Price, dtype: float64

In [32]:
df.to_csv(r"C:\Users\HP\Desktop\Python\cleaned_food_data.csv", index=False)